<a href="https://colab.research.google.com/github/shashigew/Big-Data-Project/blob/main/Country_Wise_Medal_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install PySpark
!pip install pyspark


In [2]:
# Import Required Libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max, min, count
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files


In [3]:
# Create a Spark Session
spark = SparkSession.builder \
    .appName("Seasonal Climate Pattern Analysis") \
    .getOrCreate()

spark


In [4]:
# Upload summer.csv, winter.csv, and dictionary.csv manually
print("Please upload your CSV files (summer.csv, winter.csv, dictionary.csv):")
uploaded = files.upload()


Please upload your CSV files (summer.csv, winter.csv, dictionary.csv):


Saving winter.csv to winter.csv
Saving dictionary.csv to dictionary.csv
Saving summer.csv to summer.csv


In [16]:
# Read uploaded CSVs into PySpark DataFrames
# Make sure the filenames match exactly with what you uploaded
summer_df = spark.read.csv("summer.csv", header=True, inferSchema=True)
winter_df = spark.read.csv("winter.csv", header=True, inferSchema=True)
dictionary_df = spark.read.csv("dictionary.csv", header=True, inferSchema=True)

# Display sample data
print("Summer Data Sample:")
summer_df.show(5)

print("Winter Data Sample:")
winter_df.show(5)

print("Dictionary Data Sample:")
dictionary_df.show(5)




Summer Data Sample:
+----+------+--------+----------+------------------+-------+------+--------------------+------+
|Year|  City|   Sport|Discipline|           Athlete|Country|Gender|               Event| Medal|
+----+------+--------+----------+------------------+-------+------+--------------------+------+
|1896|Athens|Aquatics|  Swimming|     HAJOS, Alfred|    HUN|   Men|      100M Freestyle|  Gold|
|1896|Athens|Aquatics|  Swimming|  HERSCHMANN, Otto|    AUT|   Men|      100M Freestyle|Silver|
|1896|Athens|Aquatics|  Swimming| DRIVAS, Dimitrios|    GRE|   Men|100M Freestyle Fo...|Bronze|
|1896|Athens|Aquatics|  Swimming|MALOKINIS, Ioannis|    GRE|   Men|100M Freestyle Fo...|  Gold|
|1896|Athens|Aquatics|  Swimming|CHASAPIS, Spiridon|    GRE|   Men|100M Freestyle Fo...|Silver|
+----+------+--------+----------+------------------+-------+------+--------------------+------+
only showing top 5 rows

Winter Data Sample:
+----+--------+--------+----------+--------------------+-------+------+

In [17]:
# Check for missing values
print("🔍 Checking for missing values...")
summer_df.select([count(when(col(c).isNull(), c)).alias(c) for c in summer_df.columns]).show()
winter_df.select([count(when(col(c).isNull(), c)).alias(c) for c in winter_df.columns]).show()

# Drop nulls (optional)
summer_df = summer_df.dropna()
winter_df = winter_df.dropna()

# Example: Convert temperature columns to float (if needed)
summer_df = summer_df.withColumn("Temperature", col("Temperature").cast("float"))
winter_df = winter_df.withColumn("Temperature", col("Temperature").cast("float"))



🔍 Checking for missing values...
+----+----+-----+----------+-------+-------+------+-----+-----+
|Year|City|Sport|Discipline|Athlete|Country|Gender|Event|Medal|
+----+----+-----+----------+-------+-------+------+-----+-----+
|   0|   0|    0|         0|      0|      4|     0|    0|    0|
+----+----+-----+----------+-------+-------+------+-----+-----+

+----+----+-----+----------+-------+-------+------+-----+-----+
|Year|City|Sport|Discipline|Athlete|Country|Gender|Event|Medal|
+----+----+-----+----------+-------+-------+------+-----+-----+
|   0|   0|    0|         0|      0|      0|     0|    0|    0|
+----+----+-----+----------+-------+-------+------+-----+-----+



AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Temperature` cannot be resolved. Did you mean one of the following? [`Sport`, `Year`, `Event`, `Gender`, `Medal`].;
'Project [Year#713, City#714, Sport#715, Discipline#716, Athlete#717, Country#718, Gender#719, Event#720, Medal#721, cast('Temperature as float) AS Temperature#1143]
+- Filter atleastnnonnulls(9, Year#713, City#714, Sport#715, Discipline#716, Athlete#717, Country#718, Gender#719, Event#720, Medal#721)
   +- Relation [Year#713,City#714,Sport#715,Discipline#716,Athlete#717,Country#718,Gender#719,Event#720,Medal#721] csv


In [18]:
# Calculate average temperature per season
summer_avg = summer_df.agg(avg("Temperature").alias("Summer_Avg_Temp")).collect()[0]["Summer_Avg_Temp"]
winter_avg = winter_df.agg(avg("Temperature").alias("Winter_Avg_Temp")).collect()[0]["Winter_Avg_Temp"]

print(f"Average Summer Temperature: {summer_avg:.2f}")
print(f"❄️ Average Winter Temperature: {winter_avg:.2f}")


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Temperature` cannot be resolved. Did you mean one of the following? [`Sport`, `Year`, `Event`, `Gender`, `Medal`].;
'Aggregate [avg('Temperature) AS Summer_Avg_Temp#1145]
+- Filter atleastnnonnulls(9, Year#713, City#714, Sport#715, Discipline#716, Athlete#717, Country#718, Gender#719, Event#720, Medal#721)
   +- Relation [Year#713,City#714,Sport#715,Discipline#716,Athlete#717,Country#718,Gender#719,Event#720,Medal#721] csv
